##SETUP

In [267]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1

# Start Scrcpy (once after PC restart)
start_scrcpy=False
if start_scrcpy:
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process


bot = bot_core.Bot()

In [2]:
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

## Game stats

In [159]:
# get stored mana 
#bot.getText(220,1360,90,50,new=False,digits=True)

# get unit mana 
bot.getText(450,1360,90,50,new=False,digits=True)

# Get upgrade mana
#bot.getText(20,1540,700,50,new=False,digits=True).split('6')


'70'

In [44]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()
#cache_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,hunter.png,364.0,2,605.0,"[0, 0]",1.0
1,icon_1,alchemist.png,200.0,4,282.0,"[0, 1]",1.0
2,icon_2,empty.png,700.0,0,0.0,"[0, 2]",1.0
3,icon_3,lightning.png,391.0,1,1683.0,"[0, 3]",1.0
4,icon_4,empty.png,999.0,0,0.0,"[0, 4]",1.0
5,icon_5,empty.png,999.0,0,0.0,"[1, 0]",1.0
6,icon_6,hunter.png,169.0,2,610.0,"[1, 1]",1.0
7,icon_7,sharpshooter.png,224.0,3,180.0,"[1, 2]",1.0
8,icon_8,hunter.png,224.0,1,1159.0,"[1, 3]",1.0
9,icon_9,empty.png,999.0,0,0.0,"[1, 4]",1.0


In [ ]:
def try_merge(prev_grid=None):
    info=''
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names,prev_grid=prev_grid)
    df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
    # check if grid full
    if ('empty.png',0) in group_keys:
        # Merge if full board
        if df_groups['empty.png']<3:
            info='Merging!'
            group_keys.remove(('empty.png',0))
            if 'hunter.png' in unit_list: unit_list.remove('hunter.png')
            for i in range(3):
                merge_tar = random.choice(unit_list)
                for j in range(3):
                    bot.merge_unit(grid_df,merge_tar)
                    time.sleep(0.2)
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        merge_tar = random.choice(unit_list)
        bot.merge_unit(grid_df,merge_tar)
    return grid_df,df_groups,info



In [363]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
group_keys.remove(('empty.png',0))
group_keys

[('hunter.png', 1),
 ('priest.png', 1),
 ('alchemist.png', 3),
 ('hunter.png', 2),
 ('lightning.png', 1),
 ('sharpshooter.png', 1),
 ('sharpshooter.png', 2),
 ('sharpshooter.png', 3)]

In [494]:
# Returns all elements which match tokens value
def filter_keys(unit_series,tokens):
    series= []
    for token in tokens:
        # check if given token is int, assume unit rank filter
        if isinstance(token,int):
            exists = unit_series.index.get_level_values('rank').isin([token]).any()
            series.append(unit_series.xs(token, level='rank',drop_level=False) if exists else None)
        else:
            series.append(unit_series.xs(token, level='unit',drop_level=False) if token in unit_series else None)
            #unit_series.loc[['priest.png']]
    return pd.concat(series)

# Select stuff to merge
merge_series = filter_keys(unit_series,[1,'priest.png'])
# drop duplicated indices
merge_series[~merge_series.index.duplicated()]
# Take random sample
merge_target =  merge_series.sample()
merge_target


unit           rank
lightning.png  1       1
Name: unit, dtype: int64

In [482]:
merge_tar.loc['priest.png']

rank
1    3
1    3
Name: unit, dtype: int64

In [485]:
merge_tar[~merge_tar.index.duplicated()]

#.index.drop_duplicates()

unit              rank
hunter.png        1       5
priest.png        1       3
lightning.png     1       1
sharpshooter.png  1       1
Name: unit, dtype: int64

# Play Game

In [40]:
bot.getScreen()

In [5]:
bot.restart_RR()

## Extra unit logic 

In [ ]:
##Extra unit logic 
# Try to merge any priestess
#if 'priest.png' in unit_list:
#    if df_groups['priest.png']<1:
#        bot.merge_unit(grid_df,'priest.png')

# Basic Loop

In [3]:
def try_merge(prev_grid=None):
    info=''
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names,prev_grid=prev_grid)
    df_split, df_groups, unit_list = bot_core.get_unit_count(grid_df)
    df_groups = df_groups.sort_values(ascending=False)
    # check if grid full
    if 'empty.png' in unit_list:
        # Merge if full board
        if df_groups['empty.png']<3:
            info='Merging!'
            unit_list.remove('empty.png')
            if 'hunter.png' in unit_list: unit_list.remove('hunter.png')
            for i in range(3):
                merge_tar = random.choice(unit_list)
                for j in range(3):
                    bot.merge_unit(grid_df,merge_tar)
                    time.sleep(0.2)
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        merge_tar = random.choice(unit_list)
        bot.merge_unit(grid_df,merge_tar)
    return grid_df,df_groups,info
    
def get_button_pos(df,button):
    button=button+'.png'
    pos=df[df['icon']==button]['pos [X,Y]'].reset_index(drop=True)[0]
    return pos

def battle_screen(start=False):
    # Scan screen for any key buttons
    button_df = bot.get_current_icons()
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    if not avail_buttons.empty:
        # list of buttons
        button_names=avail_buttons['icon'].tolist()
        if 'fighting.png' in button_names and not '0cont_button.png' in button_names:
            return avail_buttons,'fighting'
        # Start pvp if homescreen
        if start and 'pvp_button.png' in button_names and 'battle_icon.png' in button_names:
            pvp_pos = get_button_pos(button_df,'pvp_button')
            bot.click_button(pvp_pos)
            return avail_buttons, 'home'
        # Check first button is clickable
        first_button=button_names[0].split('.')[0]
        if first_button in ['back_button','battle_icon','0cont_button','1quit']:
            button_pos=avail_buttons['pos [X,Y]'].tolist()[0]
            bot.click_button(button_pos)
            return avail_buttons,'menu'
    else:
        bot.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        return button_df,'lost'
        # add recursive call to restart last recorse?
battle_screen()

(           icon  available    pos [X,Y]
 0  fighting.png       True  (669, 1359),
 'fighting')

In [4]:
wait=0
grid_df =None
# Main loop
while(1):
    output = battle_screen(start=True)
    if output[1]=='fighting':
        wait = 0
        battle_bar=trange(10)
        for i in battle_bar:
                time.sleep(0.2)
                bot.click(450,1360)
                # Upgrade units
                bot.click(500,1500,delay_mult=4)
                #bot.click(200,1500)
                #bot.click(650,1500,delay_mult=4)
                grid_df,df_groups,info = try_merge(grid_df)
                clear_output()
                display([f'{i+1}/10',output[1],info,grid_df,df_groups])
    else: 
        wait+=1
        if wait>40:
            output[1] = 'RESTARTING'
            bot.restart_RR()
        print(output[1],'wait count:',wait)

['6/10',
 'fighting',
 'Merging!',
     grid_id              unit  probability  rank  rank_error position   Age
 0    icon_0     lightning.png        280.0     1      1276.0   [0, 0]   5.0
 1    icon_1        hunter.png        170.0     1      1633.0   [0, 1]  14.0
 2    icon_2        hunter.png        182.0     1      1278.0   [0, 2]   0.0
 3    icon_3  sharpshooter.png        295.0     2       445.0   [0, 3]   0.0
 4    icon_4  sharpshooter.png        289.0     3       270.0   [0, 4]   6.0
 5    icon_5        hunter.png        246.0     1      1244.0   [1, 0]  14.0
 6    icon_6        hunter.png        169.0     1      1854.5   [1, 1]   1.0
 7    icon_7         empty.png        999.0     0         0.0   [1, 2]   6.0
 8    icon_8     alchemist.png        180.0     3       226.5   [1, 3]   0.0
 9    icon_9  sharpshooter.png        227.0     1      1628.5   [1, 4]  12.0
 10  icon_10        priest.png        180.0     1      1422.5   [2, 0]   1.0
 11  icon_11        hunter.png        231

KeyboardInterrupt: 